In [251]:
import numpy as np
import scipy as sp
import pandas as pd
import random

In [252]:
f= open('voc', 'r')
voc = f.read().strip().split("\n")

In [253]:
def parse_vocabulary(vocab):
    """Parse the vocabulary set into two dictionaries, word with its index and index with its
word. When input is null, output should be null."""
    type_to_index = {}
    index_to_type = {}
    for word in set(vocab):
        index_to_type[len(index_to_type)] = word
        type_to_index[word] = len(type_to_index)        
    return (type_to_index, index_to_type)

In [254]:
ff = open('train', 'r')
train = ff.read().strip().split("\n")
fff = open('test', 'r')
test = fff.read().strip().split("\n")

In [255]:
def parse_data(corpus):
    """Parse the corpus into a list of documents. Find out the index of each word in each document and count its number of appearance in each document. When input is null, output should be null."""
    word_ids = []
    word_cts = []     
    for document_line in corpus:
        document_word_dict = {}
        for token in document_line.split():
            if token not in type_to_index:
                continue                
            type_id = type_to_index[token]
            if type_id not in document_word_dict:
                document_word_dict[type_id] = 0
            document_word_dict[type_id] += 1

        word_ids.append(np.array(list(document_word_dict.keys())))
        word_cts.append(np.array(list(document_word_dict.values()))[np.newaxis, :])

    return (word_ids, word_cts)

In [256]:
def compute_dirichlet_expectation(dirichlet_parameter):
    """Calculate the expectation of dirichlet parameter. When input is null, output should be a warning that the input is null."""
    if not np.array(dirichlet_parameter).size:
        return ("The dirichlet_parameter is null.")
    if (len(dirichlet_parameter.shape) == 1):
        return (sp.special.psi(dirichlet_parameter)-sp.special.psi(np.sum(dirichlet_parameter)))
    return (sp.special.psi(dirichlet_parameter) - sp.special.psi(np.sum(dirichlet_parameter, 1))[:, np.newaxis])

### Tests of function parse_vocabulary, parse_data, and compute_dirichlet_expectation, for both common and edge cases. Test results are consistent with the expectations which are described in docstrings.###

In [257]:
voc_test = ["year","state","new","percent","peopl","report","million","govern","presid"]
parse_vocabulary(voc_test)

({'govern': 1,
  'million': 7,
  'new': 5,
  'peopl': 6,
  'percent': 0,
  'presid': 3,
  'report': 8,
  'state': 4,
  'year': 2},
 {0: 'percent',
  1: 'govern',
  2: 'year',
  3: 'presid',
  4: 'state',
  5: 'new',
  6: 'peopl',
  7: 'million',
  8: 'report'})

In [258]:
voc_test = []
parse_vocabulary(voc_test)

({}, {})

In [259]:
corpus_test = ["mauric adult adult peopl year h last year resolv polic polic polic polic polic polic polic motiv appear cri christian christian three troubl pride farley farley friday lock bureau gun gun gun church marino marino","work posit reach reach peopl year thought million million million presid govern year year first percent state appear abl time billion told resid complet day develop made made made made wednesday wednesday","work offer shoot thought million far year year polic polic polic polic polic polic polic polic polic cash appear abl sophist told told woman woman arrest retain found day monday citi administr prepar fame art gun"]
parse_data(corpus_test)

([array([1795, 1313, 6153, 5515, 4301, 1038, 5075, 1877, 3222, 1303, 3292,
         5342, 1439, 4193, 4391, 3504, 5812, 2398,  905, 2426, 6460, 2133]),
  array([1729, 6343, 6153, 5047, 4556, 2062, 1359, 2705, 6035, 1303, 1885,
         3622, 3432, 4009, 3948, 2799, 3635, 2098, 4723, 5812, 1654, 5111,
         1465]),
  array([2759, 6153, 1163, 3725, 2062,  973, 1303, 1307, 3292, 3748, 3622,
         1329, 3432, 3948,  882, 4974, 3773,  905, 3761, 1202, 4723, 1654,
         5807, 2234, 4093])],
 [array([[1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 7, 1, 1, 2, 1, 1, 1, 1, 3, 2, 2, 1]]),
  array([[4, 1, 3, 1, 1, 1, 1, 2, 1, 1, 1, 1, 3, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1]]),
  array([[1, 2, 1, 1, 2, 1, 1, 1, 9, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 2, 1]])])

In [260]:
corpus_test = []
parse_data(corpus_test)

([], [])

In [261]:
compute_dirichlet_expectation(eta)

array([[-6816.20562896,   -12.20339424, -6816.20562896, ...,
          -11.37191495,    -9.69662577,   -27.3306072 ],
       [-6814.90096574, -6814.90096574, -6814.90096574, ...,
        -6814.90096574,    -9.18644145,    -8.92876909],
       [-6814.98827358, -6814.98827358, -6814.98827358, ...,
        -6814.98827358,    -9.31237134,    -8.30870207],
       ..., 
       [-6815.34267505,   -10.7311724 , -6815.34267505, ...,
        -6815.34267505,    -9.13004649,   -58.52477991],
       [  -10.82413382,    -9.15102217, -6816.06867703, ...,
          -40.17127912,   -10.34264717,   -10.93796738],
       [-6815.55863679,   -14.48155799, -6815.55863679, ...,
          -18.86407708,    -9.22004585,   -29.5117594 ]])

In [262]:
eta_test = []
compute_dirichlet_expectation(eta_test)

'The dirichlet_parameter is null.'

In [263]:
def e_step(corpus=None,local_parameter_iteration=50,
local_parameter_converge_threshold=1e-6):
    """E step. When input is None, output should be document_log_likelihood, phi_sufficient_statistics, and gamma. Otherwise, it should be words_log_likelihood, gamma_values."""
    if corpus == None:
        word_ids = parsed_corpus[0]
        word_cts = parsed_corpus[1]
    else:
        word_ids = corpus[0]
        word_cts = corpus[1]
    # Initialization 
    number_of_documents = len(word_ids)
    document_log_likelihood = 0
    words_log_likelihood = 0
    phi_sufficient_statistics = np.zeros((number_of_topics, number_of_types))
    gamma_values = np.zeros((number_of_documents, number_of_topics)) + alpha_alpha[np.newaxis, :] + 1.0 * number_of_types / number_of_topics
    E_log_eta = compute_dirichlet_expectation(eta)
    if parsed_corpus != None:
        E_log_prob_eta = E_log_eta - sp.misc.logsumexp(E_log_eta, axis=1)[:, np.newaxis]

    # iterate over all documents
    for doc_id in np.random.permutation(number_of_documents):
        # compute the total number of words
        total_word_count = np.sum(word_cts[doc_id])
        # initialize gamma for this document
        gamma_values[doc_id, :] = alpha_alpha + 1.0 * total_word_count / number_of_topics

        term_ids = word_ids[doc_id]
        term_counts = word_cts[doc_id]

        # update phi and gamma until gamma converges
        for gamma_iteration in range(local_parameter_iteration):
            log_phi = E_log_eta[:, term_ids].T + np.tile(sp.special.psi(gamma_values[doc_id, :]), (word_ids[doc_id].shape[0], 1))
            log_phi -= sp.misc.logsumexp(log_phi, axis=1)[:, np.newaxis]
            gamma_update = alpha_alpha + np.array(np.sum(np.exp(log_phi + np.log(np.repeat(term_counts, number_of_topics, axis=0).T)), axis=0))
            mean_change = np.mean(abs(gamma_update - gamma_values[doc_id, :]))
            gamma_values[doc_id, :] = gamma_update
            if mean_change <= local_parameter_converge_threshold:
                break

        # compute the alpha, gamma, and phi terms
        document_log_likelihood += sp.special.gammaln(np.sum(alpha_alpha)) - np.sum(sp.special.gammaln(alpha_alpha))
        document_log_likelihood += np.sum(sp.special.gammaln(gamma_values[doc_id, :])) - sp.special.gammaln(np.sum(gamma_values[doc_id, :]))
        document_log_likelihood -= np.sum(np.dot(term_counts, np.exp(log_phi) * log_phi))

# compute the p(w_{dn} | z_{dn}, \eta) terms, which will be cancelled during M-step
        words_log_likelihood += np.sum(np.exp(log_phi.T + np.log(term_counts)) * E_log_prob_eta[:, term_ids])      
        phi_sufficient_statistics[:, term_ids] += np.exp(log_phi + np.log(term_counts.transpose())).T
        
    if corpus == None:
        gamma = gamma_values
        return (document_log_likelihood, phi_sufficient_statistics, gamma)
    else:
        return (words_log_likelihood, gamma_values)

In [264]:
def m_step(phi_sufficient_statistics):
    """M step. When input is null, output should be a warning that the input is null."""
    if not np.array(phi_sufficient_statistics).size:
        return ("The input is null.")
    # compute the beta and the eta terms
    topic_log_likelihood = number_of_topics * (sp.special.gammaln(np.sum(alpha_beta)) - np.sum(sp.special.gammaln(alpha_beta)))
    topic_log_likelihood += np.sum(np.sum(sp.special.gammaln(eta), axis=1) - sp.special.gammaln(np.sum(eta, axis=1)))

    eta_temp = phi_sufficient_statistics + alpha_beta

    # compute the sufficient statistics for alpha and update
    alpha_sufficient_statistics = sp.special.psi(gamma) - sp.special.psi(np.sum(gamma, axis=1)[:, np.newaxis])
    alpha_sufficient_statistics = np.sum(alpha_sufficient_statistics, axis=0)  

    return (topic_log_likelihood, alpha_sufficient_statistics, eta_temp)

In [265]:
# Initialization
type_to_index = parse_vocabulary(voc)[0]
index_to_type = parse_vocabulary(voc)[1]
parsed_corpus = parse_data(train)
test_corpus = parse_data(test)

number_of_topics = 100
number_of_types = len(type_to_index)
number_of_documents = len(parsed_corpus[0])

alpha_alpha = np.zeros(number_of_topics) + 1/number_of_topics
alpha_beta = np.zeros(number_of_types) + 1/number_of_types
eta = np.random.gamma(100., 1./100., (number_of_topics,number_of_types))
gamma = np.zeros((number_of_documents, number_of_topics)) + alpha_alpha[np.newaxis, :] + 1.0 * number_of_types / number_of_topics       

hyper_parameter_optimize_interval=1

### Compute likelihood and sufficient statistics. Also serve as a testing when input of e_step is null. ###

In [266]:
document_log_likelihood, phi_sufficient_statistics, gamma = e_step()

### Testing of function m_step and implement M step to fit the model.###

In [267]:
topic_log_likelihood, alpha_sufficient_statistics, eta = m_step(phi_sufficient_statistics)
phi_sufficient_statistics_test = []
m_step(phi_sufficient_statistics_test)

'The input is null.'

In [268]:
joint_log_likelihood = document_log_likelihood + topic_log_likelihood

In [269]:
def optimize_hyperparameters(alpha_sufficient_statistics, hyper_parameter_iteration=100, hyper_parameter_decay_factor=0.9, hyper_parameter_maximum_decay=10,alpha=alpha_alpha, hyper_parameter_converge_threshold=1e-6):
    """Optimize hyperparameter alpha. Since the function's input is the result of m_step function, it is unnecessary and difficult to test it. Just use it."""
    alpha_update = alpha        
    decay = 0
    for alpha_iteration in range(hyper_parameter_iteration):
        alpha_gradient = number_of_documents * (sp.special.psi(np.sum(alpha)) - sp.special.psi(alpha)) + alpha_sufficient_statistics
        alpha_hessian = -number_of_documents * sp.special.polygamma(1,alpha)

        sum_g_h = np.sum(alpha_gradient / alpha_hessian)
        sum_1_h = 1.0 / alpha_hessian
        z = number_of_documents * sp.special.polygamma(1, np.sum(alpha))
        c = sum_g_h / (1.0 / z + sum_1_h)

        # update the alpha vector
        while True:
            singular_hessian = False
            step_size = np.power(hyper_parameter_decay_factor, decay) * (alpha_gradient - c) / alpha_hessian               
            if np.any(alpha <= step_size):
                singular_hessian = True
            else: alpha_update = alpha - step_size

            if singular_hessian:
                decay += 1
                if decay > hyper_parameter_maximum_decay:
                    break
            else: break

        # check the alpha converge criteria
        mean_change = np.mean(abs(alpha_update - alpha))
        alpha = alpha_update
        if mean_change <= hyper_parameter_converge_threshold:
            break

    return (alpha)

In [270]:
alpha_alpha = optimize_hyperparameters(alpha_sufficient_statistics)
words_log_likelihood, corpus_gamma_values = e_step(corpus = test_corpus)

### For every topic, find out the words that are most likely to appear in this topic. ###

In [271]:
E_log_eta = compute_dirichlet_expectation(eta)
topic = []
for topic_index in range(number_of_topics):
    temp_list = []
    beta_probability = np.exp(E_log_eta[topic_index, :] - sp.misc.logsumexp(E_log_eta[topic_index, :]))
    i = 0
    for type_index in reversed(np.argsort(beta_probability)):
        i += 1
        if i <= 30:
            temp_list.append(index_to_type[type_index])
        else: break
    topic.append(temp_list)

index = random.sample(range(number_of_topics),10)
df = pd.DataFrame({"topic0":topic[index[9]],"topic1":topic[index[0]],"topic2":topic[index[1]],"topic3":topic[index[2]],"topic4":topic[index[3]],"topic5":topic[index[4]],"topic6":topic[index[5]],"topic7":topic[index[6]],"topic8":topic[index[7]],"topic9":topic[index[8]]})

In [272]:
df

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9
0,contract,year,new,pilot,percent,year,state,state,say,bush
1,year,state,year,eastern,year,million,polic,year,polic,percent
2,presid,report,market,year,state,percent,peopl,court,new,presid
3,govern,new,stock,judg,offici,two,say,two,rose,year
4,month,offici,price,feder,new,meet,new,new,two,state
5,new,wine,nation,day,last,month,two,judg,meet,say
6,compani,water,million,attorney,unit,report,million,say,percent,dukaki
7,ford,feeney,offici,week,peopl,fbi,fire,vote,parent,congress
8,offici,nation,dollar,time,sale,agent,unit,democrat,report,hou
9,make,law,report,record,time,govern,report,soviet,year,nation
